In [1]:
!pip install tensorflow==1.13.2

In [4]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/ColabNotebooks')

In [ ]:
from google.colab import files
files.upload() 

Saving keras_darknet19.cpython-37.pyc to keras_darknet19.cpython-37.pyc


{'keras_darknet19.cpython-37.pyc': b'B\r\r\n\x00\x00\x00\x00\xe5\x15\xa6ah\t\x00\x00\xe3\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00@\x00\x00\x00s\xae\x00\x00\x00d\x00Z\x00d\x01d\x02l\x01Z\x01d\x01d\x03l\x01m\x02Z\x02\x01\x00d\x01d\x04l\x03m\x04Z\x04m\x05Z\x05\x01\x00d\x01d\x05l\x03m\x06Z\x06\x01\x00d\x01d\x06l\x03m\x07Z\x07\x01\x00d\x01d\x07l\x08m\tZ\t\x01\x00d\x01d\x08l\nm\x0bZ\x0b\x01\x00d\td\nl\x0cm\rZ\r\x01\x00e\x02e\x04d\x0bd\x0c\x8d\x02Z\x0ee\x01\xa0\x0fe\x04\xa1\x01d\rd\x0e\x84\x00\x83\x01Z\x10d\x0fd\x10\x84\x00Z\x11d\x11d\x12\x84\x00Z\x12d\x13d\x14\x84\x00Z\x13d\x15d\x16\x84\x00Z\x14d\x17d\x18\x84\x00Z\x15d\x02S\x00)\x19z!Darknet19 Model Defined in Keras.\xe9\x00\x00\x00\x00N)\x01\xda\x07partial)\x02\xda\x06Conv2D\xda\x0cMaxPooling2D)\x01\xda\tLeakyReLU)\x01\xda\x12BatchNormalization)\x01\xda\x05Model)\x01\xda\x02l2\xe9\x02\x00\x00\x00)\x01\xda\x07compose\xda\x04same)\x01\xda\x07paddingc\x00\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00O\x00\x

In [ ]:
from google.colab import files
files.upload() 

Saving keras_yolo.cpython-37.pyc to keras_yolo.cpython-37.pyc


{'keras_yolo.cpython-37.pyc': b'B\r\r\n\x00\x00\x00\x00\xeb\x15\xa6a\x08A\x00\x00\xe3\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x14\x00\x00\x00@\x00\x00\x00s&\x01\x00\x00d\x00Z\x00d\x01d\x02l\x01Z\x01d\x01d\x02l\x02Z\x03d\x01d\x02l\x04Z\x05d\x01d\x03l\x06m\x07Z\x08\x01\x00d\x01d\x04l\tm\nZ\n\x01\x00d\x01d\x05l\tm\x0bZ\x0b\x01\x00d\x01d\x06l\x0cm\rZ\r\x01\x00d\x07d\x08l\x0em\x0fZ\x0f\x01\x00d\td\nl\x10m\x11Z\x11m\x12Z\x12m\x13Z\x13\x01\x00e\x01j\x14\xa0\x15d\x0b\xa1\x01\x01\x00e\x03\xa0\x16d\x0cd\rg\x02d\x0ed\x0fg\x02d\x10d\x11g\x02d\x12d\x13g\x02d\x14d\x15g\x02g\x05\xa1\x01Z\x17d\x16d\x17d\x18d\x19d\x1ad\x1bd\x1cd\x1dd\x1ed\x1fd d!d"d#d$d%d&d\'d(d)g\x14Z\x18d*d+\x84\x00Z\x19d,d-\x84\x00Z\x1ad.d/\x84\x00Z\x1bd0d1\x84\x00Z\x1cd2d3\x84\x00Z\x1ddBd5d6\x84\x01Z\x1ed7d8\x84\x00Z\x1fdCd:d;\x84\x01Z dDd>d?\x84\x01Z!d@dA\x84\x00Z"d\x02S\x00)Ez\x1fYOLO_v2 Model Defined in Keras.\xe9\x00\x00\x00\x00N)\x01\xda\x07backend)\x01\xda\x06Lambda)\x01\xda\x0bconcatenate)\x01\xda\x05Model\xe9\x02\x0

In [ ]:
from google.colab import files
files.upload() 

Saving __init__.cpython-37.pyc to __init__.cpython-37.pyc


{'__init__.cpython-37.pyc': b'B\r\r\n\x00\x00\x00\x00\x98\x15\xa6a\x00\x00\x00\x00\xe3\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00@\x00\x00\x00s\x04\x00\x00\x00d\x00S\x00)\x01N\xa9\x00r\x01\x00\x00\x00r\x01\x00\x00\x00r\x01\x00\x00\x00\xfaWC:\\Users\\91942\\PycharmProjects\\deepL\\deeplTENSORFLOW\\object_detection\\yad2k\\__init__.py\xda\x08<module>\x01\x00\x00\x00\xf3\x00\x00\x00\x00'}

In [ ]:
from google.colab import files
files.upload() 

Saving __init__.py to __init__ (2).py


{'__init__.py': b''}

In [ ]:
from google.colab import files
files.upload() 

Saving __init__.py to __init__ (3).py


{'__init__.py': b'from .utils import *\n'}

In [5]:
from google.colab import files
files.upload() 

Saving utils.py to utils.py


{'utils.py': b'"""Draw predicted or ground truth boxes on input image."""\nimport imghdr\nimport colorsys\nimport random\n\nimport numpy as np\nfrom PIL import Image, ImageDraw, ImageFont\nfrom tensorflow.keras import backend as K\n\nfrom functools import reduce\n\ndef preprocess_image(img_path, model_image_size):\n    image_type = imghdr.what(img_path)\n    image = Image.open(img_path)\n    resized_image = image.resize(tuple(reversed(model_image_size)), Image.BICUBIC)\n    image_data = np.array(resized_image, dtype=\'float32\')\n    image_data /= 255.\n    image_data = np.expand_dims(image_data, 0)  # Add batch dimension.\n    return image, image_data\n\ndef compose(*funcs):\n    """Compose arbitrarily many functions, evaluated left to right.\n\n    Reference: https://mathieularose.com/function-composition-in-python/\n    """\n    # return lambda x: reduce(lambda v, f: f(v), funcs, x)\n    if funcs:\n        return reduce(lambda f, g: lambda *a, **kw: g(f(*a, **kw)), funcs)\n    else:

In [ ]:
from google.colab import files
files.upload() 

Saving __init__.py to __init__ (4).py


{'__init__.py': b''}

In [ ]:
from google.colab import files
files.upload() 

Saving keras_darknet19.py to keras_darknet19 (2).py


{'keras_darknet19.py': b'"""Darknet19 Model Defined in Keras."""\nimport functools\nfrom functools import partial\n\nfrom tensorflow.keras.layers import Conv2D, MaxPooling2D\nfrom tensorflow.keras.layers import LeakyReLU\nfrom tensorflow.keras.layers import BatchNormalization\nfrom tensorflow.keras.models import Model\nfrom tensorflow.keras.regularizers import l2\n\nfrom ..utils import compose\n\n# Partial wrapper for Convolution2D with static default argument.\n_DarknetConv2D = partial(Conv2D, padding=\'same\')\n\n\n@functools.wraps(Conv2D)\ndef DarknetConv2D(*args, **kwargs):\n    """Wrapper to set Darknet weight regularizer for Convolution2D."""\n    darknet_conv_kwargs = {\'kernel_regularizer\': l2(5e-4)}\n    darknet_conv_kwargs.update(kwargs)\n    return _DarknetConv2D(*args, **darknet_conv_kwargs)\n\n\ndef DarknetConv2D_BN_Leaky(*args, **kwargs):\n    """Darknet Convolution2D followed by BatchNormalization and LeakyReLU."""\n    no_bias_kwargs = {\'use_bias\': False}\n    no_bia

In [ ]:
from google.colab import files
files.upload() 

Saving keras_yolo.py to keras_yolo (3).py


{'keras_yolo.py': b'"""YOLO_v2 Model Defined in Keras."""\nimport sys\n\nimport numpy as np\nimport tensorflow as tf\nfrom tensorflow.keras import backend as K\nfrom tensorflow.keras.layers import Lambda\nfrom tensorflow.keras.layers import concatenate\nfrom tensorflow.keras.models import Model\n\nfrom ..utils import compose\nfrom .keras_darknet19 import (DarknetConv2D, DarknetConv2D_BN_Leaky,\n                              darknet_body)\n\nsys.path.append(\'..\')\n\nvoc_anchors = np.array(\n    [[1.08, 1.19], [3.42, 4.41], [6.63, 11.38], [9.42, 5.11], [16.62, 10.52]])\n\nvoc_classes = [\n    "aeroplane", "bicycle", "bird", "boat", "bottle", "bus", "car", "cat",\n    "chair", "cow", "diningtable", "dog", "horse", "motorbike", "person",\n    "pottedplant", "sheep", "sofa", "train", "tvmonitor"\n]\n\n\ndef space_to_depth_x2(x):\n    """Thin wrapper for Tensorflow space_to_depth with block_size=2."""\n    # Import currently required to make Lambda work.\n    # See: https://github.com/fcho

In [6]:
import sys
sys.path.append('/content/gdrive/MyDrive/object_detection')

In [9]:
import utils

In [ ]:
!pip install darknet yolo


In [1]:
!git clone https://github.com/ultralytics/yolov3  # clone
%cd yolov3
#%pip install -qr requirements.txt  # install

Cloning into 'yolov3'...
remote: Enumerating objects: 9951, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 9951 (delta 19), reused 27 (delta 5), pack-reused 9858
Receiving objects: 100% (9951/9951), 9.37 MiB | 20.54 MiB/s, done.
Resolving deltas: 100% (6683/6683), done.
/content/yolov3


In [3]:
!git clone https://github.com/allanzelener/yad2k.git
%cd yad2k

fatal: destination path 'yad2k' already exists and is not an empty directory.
/content/yad2k/yad2k
ls: cannot access 'yad2k': No such file or directory


In [4]:
import keras_preprocessing

In [22]:
from utils import *

In [7]:
!pip install q keras==2.0.8

In [39]:
!pip install utils

In [1]:
import argparse
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import scipy.io
import scipy.misc
import numpy as np
import pandas as pd

import PIL
from PIL import ImageFont, ImageDraw, Image

import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization
from tensorflow.python.framework.ops import EagerTensor
from tensorflow.keras.models import load_model
from yad2k.yad2k.models.keras_yolo import yolo_head

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [35]:
import sys
sys.path.append('/content/gdrive/MyDrive/object_detection/yad2k/yad2k')

In [3]:
from yad2k.yad2k.utils.utils import  draw_boxes, get_colors_for_classes, scale_boxes, read_classes, read_anchors, preprocess_image

ImportError: ignored

In [27]:
def yolo_filter_box(boxes, box_confidence, box_class_probs, threshold = 0.6):
    box_scores = box_confidence*box_class_probs #computing box score
    box_classes = tf.math.argmax(box_scores, axis=-1) #index contaning max score
    box_class_scores = tf.math.reduce_max(box_scores,axis=-1) # max score corresponding to the box
    filtering_mask = box_class_scores >=threshold
    scores = tf.boolean_mask(box_class_scores,filtering_mask)
    boxes = tf.boolean_mask(boxes, filtering_mask)
    classes = tf.boolean_mask(box_classes, filtering_mask)
    
    return scores, boxes, classes

In [28]:
def intersection_over_union(box1, box2):
    (box1_x1, box1_y1, box1_x2, box1_y2) = box1
    (box2_x1, box2_y1, box2_x2, box2_y2) = box2
    
    # calculating the coordinates of the intersection of box1 and box2
    xi1 = max(box1[0], box2[0])
    yi1 = max(box1[1], box2[1])
    xi2 = max(box1[2], box2[2])
    yi2 = max(box1[3], box2[3])
    
    intersection_width = xi2 - xi1
    intersection_height = yi2 - yi1
    
    intersection_area = max(intersection_height,0) * max(intersection_width,0)
    
    #calculationg union aea: Union(box1,box2) = box1 + box2 - intesection(box1,box2)
    
    box1_area = (box1[3] - box1[1]) * (box1[2] - box1[0])
    box2_area = (box2[3] - box2[1]) * (box2[2] - box2[0])
    
    union_area = box1_area + box2_area -intersection_area
    
    iou = intersection_aea / union_area #iou: intersection over union.
    
    return iou
    

In [29]:
def yolo_non_max_suppressions(scores, boxes, classes, max_boxes = 10, iou_threshold = 0.5):
    
    max_boxes_tensor = tf.Variable(max_boxes, dtype='int32') #tensor that is to be used in non_max_supression.
    nms_indices = tf.image.non_max_suppression(boxes,
                                               scores,     
                                               max_boxes,
                                               iou_threshold=0.5,
                                               name=None
                                              )
    scores = tf.gather(scores, nms_indices)
    boxes = tf.gather(boxes, nms_indices)
    classes = tf.gather(classes, nms_indices)
    
    return scores, boxes, classes

In [30]:
def yolo_boxes_to_corners(box_xy, box_wh):
    #converting yolo boxe predictions to bounding boxes.
    box_mins = box_xy - (box_wh/ 2.)
    box_maxes = box_xy +(box_wh/ 2.)
    
    return tf.keras.backend.concatenate([
        box_mins[...,1:2],  #y_min
        box_mins[...,0:1],  #x_min
        box_maxes[...,1:2], #y_max
        box_maxes[...,0:1]  #x_max
    ])

In [19]:
def yolo_out(yolo_outputs, image_shape= (720,1280), max_boxes=5, score_threshold=0.6, iou_threshold=0.5):
    
    box_xy, box_wh, box_confidence, box_class_probs = yolo_outputs
    boxes = yolo_boxes_to_corners(box_xy,box_wh)
    
    scores,boxes,classes = yolo_filter_box(boxes, box_confidence, box_class_probs, score_threshold)
    
    boxes = scale_boxes(boxes,image_shape)
    
    scores, boxes, classes = yolo_non_max_suppressions(scores, boxes, classes, max_boxes=10, iou_threshold=0.5)
    
    return scores, boxes, classes

In [20]:
class_names = read_classes("/content/gdrive/MyDrive/object_detection/model_data/coco_classes.txt")
anchors = read_anchors("/content/gdrive/MyDrive/object_detection/model_data/yolo_anchors.txt")
model_image_size = (608, 608) # Same as yolo_model input layer size

NameError: ignored

In [ ]:
yolo_model = load_model('/content/gdrive/MyDrive/object_detection/model_data', compile=True)

/usr/local/lib/python3.7/dist-packages/keras/layers/core/lambda_layer.py:299: UserWarning: yad2k.models.keras_yolo is not loaded, but a Lambda layer uses it. It may cause errors.
  'function_type')
/usr/local/lib/python3.7/dist-packages/keras/layers/core/lambda_layer.py:304: UserWarning: yad2k.models.keras_yolo is not loaded, but a Lambda layer uses it. It may cause errors.
  'output_shape_type')


In [ ]:
yolo_model.summary()


Model: "functional_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 608, 608, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 608, 608, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 608, 608, 32  128        ['conv2d[0][0]']                 
 alization)                     )                                                      

In [ ]:
def predections(image_file):
    
    image, image_data = preprocess_image('/content/gdrive/MyDrive/object_detection/images/' + image_file, model_image_size=(608,608))
    
    yolo_model_outputs = yolo_model(image_data)
    yolo_outputs = yolo_head(yolo_model_outputs, anchors, len(class_names))
    
    out_scores, out_boxes, out_classes = yolo_out(yolo_outputs, [image.size[1], image.size[0]], 10,0.3,0.5)
    
    print('Found {} boxes for {}'.format(len(out_boxes), '/content/gdrive/MyDrive/object_detection/images/' + image_file))
    colors = get_colors_for_classes(len(class_names))
    draw_boxes(image, out_boxes, out_classes, class_names, out_scores)
    image.save(os.path.join('/content/gdrive/MyDrive/object_detection/out',image_file), quality=100)
    
    output_image = Image.open(os.path.join('/content/gdrive/MyDrive/object_detection/out', image_file))
    imshow(output_image)
    
    return out_scores, out_boxes, out_classes


In [ ]:
out_scores, out_boxes, out_classes = predections('test.jpg')

NameError: ignored